In [1]:

# В Контуре есть 4 продукта для ритейла и товарного учета:
# 〰️Контур.Ритейл
# 〰️Контур.Поставки
# 〰️Контур.Маркет
# 〰️Контур.ОФД   

# Во всех четырёх идёт работа с товарами. Компания хочет, чтобы:
# ◾️ названия товаров были понятными и читаемыми;
# ◾️ были удобные классификаторы, с помощью которых можно было выбирать один или множество товаров:
#  ▫️ категория товара (чай, творог, и др.);
#  ▫️ объём/вес/количество товара в упаковке;
#  ▫️ рецепт товара (провансаль, пломбир, и др.);
#  ▫️ тип упаковки (банка, бутылка, коробка и др.).

#### Вам предлагается попробовать классифицировать тип упаковки товара по его названию.

## 🗂Формат входных данных
# Датасет разделен на две части:
# 🔸 train – для обучения, в нём присутствует ID, название товара (name) и тип его упаковки (tare);
# 🔸 test – для оценки качества обученного вами классификатора, в нём присутствует только ID и название товара.
# Метрика качества, которая применяется для оценки: accuracy.

## ✔️Ожидаемый результат
# Требуется обучить модель на данных из train и предсказать тип упаковки для данных из test. 
# Нужно сформировать 2 файла:
# 🔸 prediction.csv: предсказание для test в формате csv – id,tare;
# 🔸 report.ipynb: ноутбук с описанием хода вашего решения от изучения данных до построения предсказания для test.csv.
# 

In [110]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
import scikitplot as skplt
from sklearn.preprocessing import MinMaxScaler, StandardScaler


[nltk_data] Downloading package punkt to /Users/shu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/shu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_train_set = pd.read_csv('train.csv')  

In [4]:
df_train_set.head()

,id,name,tare
0,0,Котлеты МЛМ из говядины 335г,коробка
1,1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


In [5]:
df_test_set = pd.read_csv('test.csv')

In [6]:
df_test_set.head()

,id,name
0,40648,"Масло ""Чишминское"" халяль рафинированное 1 л"
1,40649,"Печенье""Танец бабочки""вес Глобус"
2,40650,Крупа МЕТАКА пшенич 5*100
3,40651,Конфеты МИННИ-УХ 200г Сладуница
4,40652,"Хейлис Королевский Слон чай черный байховый 1,..."


In [7]:
df_test_set.drop(columns='id', inplace=True)

In [8]:
df_train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40648 entries, 0 to 40647
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      40648 non-null  int64 
 1   name    40648 non-null  object
 2   tare    40648 non-null  object
dtypes: int64(1), object(2)
memory usage: 952.8+ KB


In [9]:
df_train_set.drop_duplicates(inplace=True)

In [10]:
df_train_set.drop(columns='id', inplace=True)

In [11]:
df_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40648 entries, 0 to 40647
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    40648 non-null  object
 1   tare    40648 non-null  object
dtypes: object(2)
memory usage: 952.7+ KB


In [12]:
df_train_set.head()

,name,tare
0,Котлеты МЛМ из говядины 335г,коробка
1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


In [13]:
df_train_set.tare.value_counts(normalize=True).round(3) * 100

пакет без формы                   22.2
бутылка                           18.4
коробка                           10.3
пакет прямоугольный                8.6
обертка                            7.9
банка неметаллическая              5.5
стаканчик                          5.1
банка металлическая                4.5
вакуумная упаковка                 2.6
усадочная упаковка                 2.4
контейнер                          2.2
пачка                              1.7
лоток                              1.5
туба                               1.4
гофрокороб                         1.0
колбасная оболочка                 1.0
тортница                           0.8
без упаковки                       0.8
упаковка с газовым наполнением     0.7
ведро                              0.6
ячеистая упаковка                  0.6
Name: tare, dtype: float64

In [14]:
for i, name_tare in enumerate(sorted(df_train_set.tare.value_counts().index)):
    df_train_set.loc[df_train_set.tare == name_tare, 'range'] = i

In [15]:
# df_train_set.drop(columns=['range'], inplace=True)

In [16]:
df_train_set.shape

(40648, 3)

In [17]:
df_train_set.head()

,name,tare,range
0,Котлеты МЛМ из говядины 335г,коробка,9.0
1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка,9.0
2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик,15.0
3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер,8.0
4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы,12.0


In [18]:
df_train_set.range = df_train_set.range.astype('int')

In [19]:
df_train_set.head()

,name,tare,range
0,Котлеты МЛМ из говядины 335г,коробка,9
1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка,9
2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик,15
3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер,8
4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы,12


In [20]:
df_train_set.shape

(40648, 3)

In [22]:
df_train, df_test = train_test_split(df_train_set.drop(columns='tare'), test_size=0.20)

In [23]:
df_train.shape

(32518, 2)

In [24]:
df_test.shape

(8130, 2)

In [67]:
s_example = df_train_set.iloc[1]['name']
tokens = word_tokenize(s_example, language='russian')
tokens_w_punctuation = [i for i in tokens if i not in string.punctuation]
rus_stopwords = stopwords.words("russian")
tokens_w_punct_and_stopword = [i for i in tokens_w_punctuation if i not in rus_stopwords]
snowball = SnowballStemmer(language='russian')
stemmed_tokens = [snowball.stem(i) for i in tokens_w_punct_and_stopword]

In [68]:
print(f"Исходный текст: {s_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_w_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_w_punct_and_stopword}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕДА):20
-----------------
Токены: ['Победа', 'Вкуса', 'конфеты', 'Мишки', 'в', 'лесу', '250г', '(', 'КФ', 'ПОБЕДА', ')', ':20']
-----------------
Токены без пунктуации: ['Победа', 'Вкуса', 'конфеты', 'Мишки', 'в', 'лесу', '250г', 'КФ', 'ПОБЕДА', ':20']
-----------------
Токены без пунктуации и стоп слов: ['Победа', 'Вкуса', 'конфеты', 'Мишки', 'лесу', '250г', 'КФ', 'ПОБЕДА', ':20']
-----------------
Токены после стемминга: ['побед', 'вкус', 'конфет', 'мишк', 'лес', '250г', 'кф', 'побед', ':20']
-----------------


In [69]:
rus_stopwords = stopwords.words("russian")
snowball = SnowballStemmer(language='russian')


def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language='russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in rus_stopwords]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [70]:
tokenize_sentence(s_example)

['побед', 'вкус', 'конфет', 'мишк', 'лес', '250г', 'кф', 'побед', ':20']

In [71]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)

In [72]:
features = vectorizer.fit_transform(df_train.name)

In [82]:
model = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000)
model.fit(features, df_train.range)

LogisticRegression(max_iter=1000, random_state=0)

In [83]:
model.predict(features[25])

array([13])

In [84]:
df_train.iloc[25]

name     Барби Стильные Браслеты И Жевательная Конфета 11г
range                                                   13
Name: 14634, dtype: object

In [89]:
model_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('model', model)
])

In [90]:
model_pipeline.fit(df_train.name, df_train.range)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x281c25d00>)),
                ('model', LogisticRegression(max_iter=1000, random_state=0))])

In [92]:
pred = model_pipeline.predict(df_test.name)

In [93]:
pred

array([12, 12, 12, ...,  3, 16, 12])

In [94]:
precision_score(y_true=df_test.range, y_pred=pred, average='micro')

0.8115621156211562

In [97]:
recall_score(y_true=df_test.range, y_pred=pred, average='micro')

0.8115621156211562

In [122]:
predict = model_pipeline.predict(df_test_set.name)

In [123]:
predict

array([ 3,  6,  9, ..., 12, 12,  3])

In [126]:
df_test_set.head()

,name
0,"Масло ""Чишминское"" халяль рафинированное 1 л"
1,"Печенье""Танец бабочки""вес Глобус"
2,Крупа МЕТАКА пшенич 5*100
3,Конфеты МИННИ-УХ 200г Сладуница
4,"Хейлис Королевский Слон чай черный байховый 1,..."


In [127]:
df_test_set['range'] = predict

In [132]:
df_test_set

,name,range
0,"Масло ""Чишминское"" халяль рафинированное 1 л",3
1,"Печенье""Танец бабочки""вес Глобус",6
2,Крупа МЕТАКА пшенич 5*100,9
3,Конфеты МИННИ-УХ 200г Сладуница,12
4,"Хейлис Королевский Слон чай черный байховый 1,...",9
...,...,...
19790,Чай БЕТА Чемпион Long leaf 100 г=50,9
19791,100Г МАССА РОСТОВСКАЯ АССОРТИ,9
19792,Крупа УВЕЛКА ячневая 600г,12
19793,Говядина мраморная зерновой откорм Гуляш из го...,12


In [133]:
for i, name_tare in enumerate(sorted(df_train_set.tare.value_counts().index)):

    df_test_set.loc[df_test_set.range == i, 'tare'] = name_tare

In [134]:
df_test_set

,name,range,tare
0,"Масло ""Чишминское"" халяль рафинированное 1 л",3,бутылка
1,"Печенье""Танец бабочки""вес Глобус",6,гофрокороб
2,Крупа МЕТАКА пшенич 5*100,9,коробка
3,Конфеты МИННИ-УХ 200г Сладуница,12,пакет без формы
4,"Хейлис Королевский Слон чай черный байховый 1,...",9,коробка
...,...,...,...
19790,Чай БЕТА Чемпион Long leaf 100 г=50,9,коробка
19791,100Г МАССА РОСТОВСКАЯ АССОРТИ,9,коробка
19792,Крупа УВЕЛКА ячневая 600г,12,пакет без формы
19793,Говядина мраморная зерновой откорм Гуляш из го...,12,пакет без формы
